In [ ]:
#default_exp anime.dataloading

In [ ]:
#export
import pandas as pd
from pathlib import Path
import json
from functools import partial
from PIL import Image
import numpy as np
from itertools import chain
from fastai2.torch_basics import *
from fastai2.layers import *
from fastai2.data.all import *
from fastai2.data.block import *
from fastai2.optimizer import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from anime.ugatit import *
from anime.ugatit_networks import ILN,adaILN
from torch.autograd import Variable
import math

Currently not really used, will be for alternative dataset

In [ ]:
path=Path("/home/fast/.fastai/data/danbooru2018")
meta_path=path/'metadata/2018000000000016'
portraits=pd.read_pickle(path/'portraits')
selfie_path=Path('/home/fast/.fastai/data/Selfie-dataset')
self_txt= selfie_path/'selfie_dataset.txt'
attrs='image_name popularity partial_faces is_female baby child teenager youth middle_age senior white black asian oval_face round_face heart_face smiling mouth_open frowning wearing_glasses wearing_sunglasses wearing_lipstick tongue_out duck_face black_hair blond_hair brown_hair red_hair curly_hair straight_hair braid_hair showing_cellphone using_earphone using_mirror braces wearing_hat harsh_lighting dim_lighting'.split()

In [ ]:
#export
def _default_ugatit_datastats():
        data_stats={'selfie_means':torch.Tensor([143.81911227601947, 119.46716940677527, 113.92494676532168]),
         'selfie_stds':torch.Tensor([67.72407215465199, 64.41496156519109, 52.516515323610015]),
         'anime_means':torch.Tensor([173.24273681640625, 155.69161987304688, 135.62557983398438]),
         'anime_stds':torch.Tensor([51.790767669677734, 48.620208740234375, 48.380496978759766])}
        return {'means':(data_stats['selfie_means']+data_stats['anime_means'])/2,
             'stds':(data_stats['selfie_stds']+data_stats['anime_stds'])/2,
             'real_stats':data_stats}

## Tag comparison

These are currently not used, but can be used to access dataframes that other _danbooru dataset was generated from

In [ ]:
def getSelfiePath(df,dir,i):
    dft=df.iloc[[i]]
    path=dir/((dft['image_name']).values[0]+'.jpg')
    return (dft.values,path)
def getImagePath(df,dir,i):
    dft=df.iloc[[i]]
    path=dir/'512px'/(dft['directory']+'/'+dft['file']).values[0]
    return (dft.tags.values[0],path)

In [ ]:
#export
class DSList():
    def __init__(self,name='selfie2anime'):
        self.name=name
    def __call__(self):
        if(self.name=='selfie2anime'):
            return self._selfie2anime()
        elif(self.name=='_danbooru'):
            return self._danbooru()
        elif(self.name=='selfie2animeTest'):
            return self._selfie2animeTest()
    def _danbooru(self):
        def txt_to_path(img,dir=dir,ext='.jpg'):
            return dir/(img+ext)
        selfies=pd.read_pickle(selfie_path/'selfies')
        selfie_imgs=L(list(selfies['image_name'].apply(partial(txt_to_path,dir=selfie_path/'images'))))
        f=partial(txt_to_path,dir=path/'512px',ext='')
        locs=portraits['directory']+'/'+portraits['file']
        anime_imgs=L(list(locs.apply(f)))
        return selfie_imgs,anime_imgs
    def _selfie2anime(self):
        selfie_imgs=L(Path("/home/fast/.fastai/data/selfie2anime/trainA").ls())
        anime_imgs=L(Path("/home/fast/.fastai/data/selfie2anime/trainB").ls())
        return selfie_imgs,anime_imgs
    def _selfie2animeTest(self):
        selfie_imgs=L(Path("/home/fast/.fastai/data/selfie2anime/testA").ls())
        anime_imgs=L(Path("/home/fast/.fastai/data/selfie2anime/testB").ls())
        return selfie_imgs,anime_imgs

In [ ]:
selfies,animes=DSList()()
assert(len(animes)==3400)
assert(len(selfies)==3400)

In [ ]:
#export
denorm = lambda x,data_stats: (x*data_stats['stds'][...,None,None]+data_stats['means'][...,None,None])/255

In [ ]:
#export
class CycleImage(Tuple):
    def toTensor(self):
        selfie,anime = self
        return torch.cat([selfie,anime], dim=2)
    def show(self, ctx=None,data_stats=_default_ugatit_datastats(), **kwargs): 
        img=self.toTensor().detach().cpu()
        half=int(img.shape[2]/2)
        selfie=denorm(img[:,:,0:half],data_stats)
        anime=denorm(img[:,:,half:],data_stats)
        return show_image(torch.cat([selfie,anime], dim=2),  ctx=ctx)

In [ ]:
#export
class CyclePair(Transform):
    def __init__(self,bItems):
        self.small = bItems 
        self.assoc = self
        
    def encodes(self,i):
        "x: tuple of `i`th image and a random image from same or different class; y: True if same class"
        return CycleImage(i, random.choice(self.small))

In [ ]:
#export
class ToTensorTuple(Transform):
    "Convert item to appropriate tensor class"
    order = 15
    def encodes(self, o:CycleImage): return TensorImage(image2byte(o))

In [ ]:
#export
def resized_image(fn:Path, sz=128):
    x = Image.open(fn).resize((sz,sz))
    x = array(x)
    if x.ndim==3:
        return tensor(x).permute(2,0,1).float()/255.
    elif x.ndim==2:
        return tensor(x)[None].repeat(3,1,1).float()/255.

In [ ]:
#export
class ListofTransformsforTuple(Transform):
    "Convert item to appropriate tensor class"
    order = 60
    def __init__(self, transforms, stats,norm):
        self.transforms=transforms
        self.stats=to_device(stats)
        self.norm=norm
    def encodes(self, o): 
        if(o.shape[-3:]==torch.Size([1, 1, 1])): #don't transform the labels
            return o
        o=TensorImage(o)
        with torch.no_grad():
            for t in self.transforms:
                o=t(o)
            o = self.norm(to_device(self.stats,device=o.device),o)
        return o

In [ ]:
#export
class NoopTensor(Transform):
    def encodes(self, x): return CycleImage(torch.Tensor([[[0]]]),torch.Tensor([[[0]]]))

In [ ]:
#export
@typedispatch
def show_batch(x:CycleImage, y:CycleImage, samples, ctxs=None, max_n=9, **kwargs):
    if ctxs is None: ctxs = Inf.nones
    for i in range_of(samples[0]):
        [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i),ctxs,range(max_n))]
    return ctxs

In [ ]:
#export
class UGATITData():
    def __init__(self,selfie_list=DSList()()[0],anime_list=DSList()()[1],img_size=256):
        self.selfie_list,self.anime_list=selfie_list,anime_list
        self.data_stats=_default_ugatit_datastats()
        self.img_size=img_size
    def create_ugatit_dls(self,sample_size=3400,val=None,bs=3,augmentations=aug_transforms()):
        val=ifnone(val,['female_10686.jpg','female_10367.jpg','female_30988.jpg','female_8194.jpg','female_18724.jpg','female_10087.jpg','female_27647.jpg','female_13409.jpg','female_20608.jpg','female_26279.jpg','female_7103.jpg','female_10615.jpg','female_25884.jpg','female_31235.jpg','female_28991.jpg','female_9936.jpg','female_33285.jpg','female_33491.jpg','female_1597.jpg', 'female_13893.jpg', 'female_26481.jpg', 'female_32384.jpg', 'female_18827.jpg', 'female_19176.jpg', 'female_6150.jpg','female_1487.jpg', 'female_13914.jpg', 'female_29264.jpg', 'female_17571.jpg', 'female_32817.jpg', 'female_7374.jpg', 'female_16798.jpg', 'female_21739.jpg', 'female_22605.jpg', 'female_21305.jpg', 'female_12988.jpg', 'female_4297.jpg', 'female_9025.jpg', 'female_13560.jpg', 'female_31712.jpg', 'female_1177.jpg', 'female_13787.jpg', 'female_11117.jpg', 'female_10664.jpg', 'female_17134.jpg', 'female_16420.jpg', 'female_95.jpg', 'female_15298.jpg', 'female_32776.jpg', 'female_9353.jpg', 'female_22181.jpg', 'female_32015.jpg', 'female_27565.jpg', 'female_19001.jpg', 'female_8955.jpg', 'female_33094.jpg', 'female_7486.jpg', 'female_32167.jpg', 'female_28716.jpg', 'female_6519.jpg', 'female_4339.jpg', 'female_24557.jpg', 'female_20786.jpg', 'female_26695.jpg', 'female_25546.jpg', 'female_15424.jpg', 'female_711.jpg', 'female_31004.jpg', 'female_9440.jpg', 'female_22912.jpg', 'female_2754.jpg', 'female_24839.jpg', 'female_12741.jpg', 'female_9480.jpg', 'female_15677.jpg', 'female_23557.jpg', 'female_3006.jpg', 'female_29858.jpg', 'female_8664.jpg', 'female_16808.jpg', 'female_32694.jpg', 'female_5019.jpg', 'female_15584.jpg', 'female_32256.jpg', 'female_5747.jpg', 'female_22101.jpg', 'female_4115.jpg', 'female_20185.jpg', 'female_25131.jpg', 'female_31061.jpg', 'female_2546.jpg', 'female_30868.jpg', 'female_17341.jpg', 'female_9415.jpg', 'female_14142.jpg', 'female_11470.jpg', 'female_683.jpg', 'female_3433.jpg', 'female_19286.jpg', 'female_30070.jpg'])
        tfms = Transform(partial(resized_image,sz=self.img_size))
        tfms = Pipeline([CyclePair(self.anime_list), tfms, IntToFloatTensor(div=False)])
        noop = Pipeline([NoopTensor()])
        val_path=self.selfie_list[0].parent
        sample_size=len(self.selfie_list)-100 if sample_size<3 or sample_size>len(self.selfie_list)-100 else sample_size
        selfie_sample=self.selfie_list.filter(lambda s:s.name in val,negate=True)
        selfie_sample=selfie_sample.shuffle()[0:sample_size]
        selfie_sample=selfie_sample+map(lambda v:val_path/v,val)
        tfmDs=Datasets(selfie_sample,[tfms,noop],splits=[[selfie.name not in val for selfie in selfie_sample],[selfie.name in val for selfie in selfie_sample]])
        augmentation=augmentations
        augmentation=ListofTransformsforTuple(augmentation, self.data_stats, self.norm)
        return tfmDs.dataloaders(bs=bs,val_bs=1,after_batch=[augmentation], num_workers=0)
    def norm(self,stats,x): return (255*x-unsqueeze(stats['means'],n=2)[None])/unsqueeze(stats['stds'],n=2)[None]

In [ ]:
dls=UGATITData().create_ugatit_dls()

In [ ]:
dls.one_batch()

In [ ]:
dls.show_batch()

In [ ]:
dls.valid.one_batch()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()